## Bursts processing on demand

### Introduction
Sentinel-1 Interferometric Wide (IW) mode and Extra-Wide Swath (EW) mode single look complex (SLC) data are collected using a form of ScanSAR imaging called Terrain Observation with Progressive Scans SAR (TOPSAR). 
In these modes, data is collected by cyclically observing, then switching the antenna beam between multiple adjacent sub-swaths. 
Within each observation of a sub-swath, a short sequence of radar pulses are transmitted in rapid succession, imaging a contiguous portion of the target surface. 
These short sequences of radar pulses are called “bursts”, and are the fundamental unit of SLC data collected using the TOPSAR technique.

Data from many bursts are packaged together to create the standard Sentinel-1 IW and EW SLC products. 
These products are produced by creating sub-swath images that contain a series of overlapping bursts, where each burst has been processed as a separate SLC image. 
Sub-swath images from all available swaths (three swaths for IW mode and five swaths for EW mode) are packaged together to create the final SLC products. 
While data from many bursts are packaged together in the standard SLC products, SLC data from a single burst can be utilized independently.

### Bursts processing service
Bursts processing service allows you to extract single burst image from the SLC product. In order to do so, run the following steps.

## Step 1: Define helper functions and import necessary modules

In [ ]:
import getpass
import uuid
from pathlib import Path

import requests
import urllib3

urllib3.disable_warnings()


TOKEN_URL = "https://identity.dataspace.copernicus.eu/auth/realms/cdse/protocol/openid-connect/token"
CLIENT_ID = "cdse-public"


def get_token(username: str, password: str) -> str:
    assert username, "Username is required!"
    assert password, "Password is required!"

    response = requests.post(
        TOKEN_URL,
        data={
            "client_id": CLIENT_ID,
            "username": username,
            "password": password,
            "grant_type": "password",
        },
    )
    response.raise_for_status()

    access_token = response.json()["access_token"]
    print("Acquired keycloak token!")

    return access_token


def process_burst(burst_id: str, token: str, output_dir: Path) -> None:
    assert burst_id, "Burst ID is required!"
    assert token, "Keycloak token is required!"

    print("Processing burst...")

    response = requests.post(
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts({burst_id})/$value",
        headers={"Authorization": f"Bearer {token}"},
        verify=False,
        allow_redirects=False,
        stream=True,
    )

    if 300 <= response.status_code < 400:
        redirect_url = response.headers["Location"]
        response = requests.post(
            redirect_url, headers={"Authorization": f"Bearer {token}"}, verify=False, stream=True, allow_redirects=False
        )

    if response.status_code != 200:
        err_msg = response.json() if response.headers.get("Content-Type") == "application/json" else response.text
        raise RuntimeError(f"Failed to process burst: \n{err_msg}")

    print("Processing has been successful!")

    try:
        zipfile_name = response.headers["Content-Disposition"].split("filename=")[1]
    except (KeyError, IndexError):
        zipfile_name = "output_burst.zip"

    output_path = output_dir / zipfile_name
    print("Saving output product...")
    with open(output_path, "wb") as target_file:
        for chunk in response.iter_content(chunk_size=8192):
            target_file.write(chunk)
    print("Output product has been saved to:", output_path)

## Step 2: Define user credentials

In [ ]:
USERNAME = input("Enter your CDSE username:")
PASSWORD = getpass.getpass(prompt='Enter your CDSE Password: ', stream=None)

TOKEN = get_token(username=USERNAME, password=PASSWORD)

## Step 3: Define burst_id
(can be found in catalogue https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$orderby=ContentDate/Start%20desc - field called Id)

In [ ]:
BURST_ID = input("Enter burst uuid (from catalogue):")

try:
    uuid.UUID(BURST_ID)
except ValueError:
    raise ValueError("Burst ID is not a valid UUID!")

response = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts({BURST_ID})")

if response.status_code == 404:
    raise ValueError(f"Burst with uuid '{BURST_ID}' not found in catalogue!")

response.raise_for_status()

## Step 4: Define directory where the produced burst zip should be stored (if left empty the product will be saved in working directory)

In [ ]:
output_dir = input("OUTPUT_DIR:")
OUTPUT_DIR = Path(output_dir)

if not OUTPUT_DIR.exists():
    try:
        OUTPUT_DIR.mkdir(exists_ok=True, parents=True)
    except Exception as err:
        raise FileNotFoundError("Could not find nor create output directory") from err

## Final step: Run burst processing and await output

In [ ]:
process_burst(burst_id=BURST_ID, token=TOKEN, output_dir=OUTPUT_DIR)